# Criando DataFrames
Vamos criar objetos `DataFrame` a partir de outras estruturas de dados em Python, lendo um arquivo CSV e consultando um banco de dados.

## Sobre os Dados
Neste notebook, estaremos trabalhando com dados de terremotos de 18 de setembro de 2018 a 13 de outubro de 2018 (obtidos do Serviço Geológico dos Estados Unidos (USGS) usando a [API USGS](https://earthquake.usgs.gov/fdsnws/event/1/))

## Imports

In [1]:
import datetime as dt
import numpy as np
import pandas as pd

## Criando um objeto `Series`

In [2]:
np.random.seed(0)  # set a seed for reproducibility
pd.Series(np.random.rand(5), name='random')

0    0.548814
1    0.715189
2    0.602763
3    0.544883
4    0.423655
Name: random, dtype: float64

## Criando um objeto `DataFrame` a partir de um objeto `Series`
Usando o método `to_frame()`:

In [3]:
pd.Series(np.linspace(0, 10, num=5)).to_frame()

,0
0,0.0
1,2.5
2,5.0
3,7.5
4,10.0


## Criando um `DataFrame` a partir de Estruturas de Dados em Python
### De um dicionário de estruturas semelhantes a listas
Os valores do dicionário podem ser listas, arrays do NumPy, etc., contanto que tenham comprimento (geradores não têm comprimento, então não podemos usá-los aqui):

In [5]:
np.random.seed(0)  # set seed so result is reproducible
pd.DataFrame(
    {
        'random': np.random.rand(5),
        'text': ['hot', 'warm', 'cool', 'cold', None],
        'truth': [np.random.choice([True, False]) for _ in range(5)]
    },
    index=pd.date_range(
        end=dt.date(2024, 6, 11),
        freq='1D',
        periods=5,
        name='date'
    )
)

,random,text,truth
date,,,
2024-06-07,0.548814,hot,False
2024-06-08,0.715189,warm,True
2024-06-09,0.602763,cool,True
2024-06-10,0.544883,cold,False
2024-06-11,0.423655,None,True


### De uma lista de dicionários

In [6]:
pd.DataFrame([
    {'mag': 5.2, 'place': 'California'},
    {'mag': 1.2, 'place': 'Alaska'},
    {'mag': 0.2, 'place': 'California'},
])

,mag,place
0,5.2,California
1,1.2,Alaska
2,0.2,California


### A partir de uma lista de tuplas

In [7]:
list_of_tuples = [(n, n**2, n**3) for n in range(5)]
list_of_tuples

[(0, 0, 0), (1, 1, 1), (2, 4, 8), (3, 9, 27), (4, 16, 64)]

In [8]:
pd.DataFrame(
    list_of_tuples,
    columns=['n', 'n_squared', 'n_cubed']
)

,n,n_squared,n_cubed
0,0,0,0
1,1,1,1
2,2,4,8
3,3,9,27
4,4,16,64


### A partir de um NumPy array

In [9]:
pd.DataFrame(
    np.array([
        [0, 0, 0],
        [1, 1, 1],
        [2, 4, 8],
        [3, 9, 27],
        [4, 16, 64]
    ]), columns=['n', 'n_squared', 'n_cubed']
)

,n,n_squared,n_cubed
0,0,0,0
1,1,1,1
2,2,4,8
3,3,9,27
4,4,16,64


## Criando um objeto `DataFrame` a partir do conteúdo de um arquivo CSV

### Encontrando informações sobre o arquivo antes de lê-lo
Antes de tentar ler um arquivo, podemos usar o comando de linha para ver informações importantes sobre o arquivo que podem determinar como o lemos. Podemos executar código de linha de comando em Notebooks Jupyter (graças ao IPython) usando `!` antes do código.

#### Número de linhas (contagem de linhas)
Por exemplo, podemos descobrir quantas linhas há no arquivo usando o utilitário `wc` (contagem de palavras) e contando as linhas no arquivo (`-l`). O arquivo possui 9.333 linhas:

In [10]:
!wc -l data/earthquakes.csv

    9333 data/earthquakes.csv


**Usuários do Windows**: se o comando acima não funcionar para você (depende da sua configuração), use este em vez disso:

```bash
!find /c /v "" data\earthquakes.csv
```

#### Tamanho do arquivo
Podemos encontrar o tamanho do arquivo usando `ls` para listar os arquivos no diretório `data`, e passando as opções `-lh` para incluir o tamanho do arquivo em um formato legível para humanos. Em seguida, usamos `grep` para encontrar o arquivo em questão. Observe que `|` passa o resultado de `ls` para `grep`. O utilitário `grep` é usado para encontrar itens que correspondem a padrões.

Isso nos informa que o arquivo tem 3,4 MB:

In [11]:
!ls -lh data | grep earthquakes.csv

-rw-r--r--  1 goliveira  staff   3,4M  1 Jun 14:56 earthquakes.csv


**Usuários do Windows**: se o comando acima não funcionar para você (depende da sua configuração), use este em vez disso:

```bash
!dir data | findstr "earthquakes.csv"
```

Podemos até mesmo capturar o resultado de um comando e usá-lo no nosso código Python:

In [12]:
files = !ls -lh data
[file for file in files if 'earthquake' in file]

['-rw-r--r--  1 goliveira  staff   3,4M  1 Jun 14:56 earthquakes.csv']

**Usuários do Windows**: se o comando acima não funcionar para você (depende da sua configuração), use este em vez disso:

```python
files = !dir data
[file for file in files if 'earthquake' in file]
```

#### Examinando algumas linhas
Podemos usar `head` para ver as primeiras `n` linhas do arquivo. Com a opção `-n`, podemos especificar quantas. Isso nos mostra que a primeira linha do arquivo contém cabeçalhos e que ele é separado por vírgulas (apenas porque a extensão do arquivo é `.csv` não significa que contém valores separados por vírgulas):

In [15]:
!head -n 2 data/earthquakes.csv

alert,cdi,code,detail,dmin,felt,gap,ids,mag,magType,mmi,net,nst,place,rms,sig,sources,status,time,title,tsunami,type,types,tz,updated,url
,,37389218,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci37389218&format=geojson,0.008693,,85.0,",ci37389218,",1.35,ml,,ci,26.0,"9km NE of Aguanga, CA",0.19,28,",ci,",automatic,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475395144,https://earthquake.usgs.gov/earthquakes/eventpage/ci37389218


**Usuários do Windows**: se o comando acima não funcionar para você (depende da sua configuração), use este em vez disso:

```python
n = 2
with open('data/earthquakes.csv', 'r') as file:
    for _ in range(n):
        print(file.readline(), end='\r')
```

Assim como `head` mostra as primeiras linhas, `tail` mostra as últimas linhas. Isso pode nos ajudar a verificar se não há dados extras no final do arquivo, como talvez metadados sobre os campos que na verdade não fazem parte do conjunto de dados:

In [16]:
!tail -n 1 data/earthquakes.csv

,,38063935,https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci38063935&format=geojson,0.01698,,39.0,",ci38063935,",0.66,ml,,ci,24.0,"9km NE of Aguanga, CA",0.1,7,",ci,",reviewed,1537228864470,"M 0.7 - 9km NE of Aguanga, CA",0,earthquake,",focal-mechanism,geoserve,nearby-cities,origin,phase-data,scitech-link,",-480.0,1537305830770,https://earthquake.usgs.gov/earthquakes/eventpage/ci38063935


**Usuários do Windows**: se o comando acima não funcionar para você (depende da sua configuração), use este em vez disso:

```python
import os

with open('data/earthquakes.csv', 'rb') as file:
    file.seek(0, os.SEEK_END)
    while file.read(1) != b'\n':
        file.seek(-2, os.SEEK_CUR)
    print(file.readline().decode())
```

*Nota*: Para inspecionar mais de uma linha do final do arquivo, você terá que usar o seguinte, que requer ler o arquivo inteiro:

```python
n = 2
with open('data/earthquakes.csv', 'r') as file:
    print('\r'.join(file.readlines()[-n:]))
```

#### Contagem de colunas
Podemos usar `awk` para encontrar a contagem de colunas. Este é um utilitário para escaneamento e processamento de padrões. A opção `-F` nos permite especificar o delimitador (vírgula, neste caso). Em seguida, especificamos o que fazer para cada registro no arquivo. Escolhemos imprimir `NF`, que é uma variável predefinida cujo valor é o número de campos no registro atual. Aqui, usamos `exit` para que imprimamos o número de campos na primeira linha do arquivo.

Isso nos diz que temos 26 colunas de dados:

In [17]:
!awk -F',' '{print NF; exit}' data/earthquakes.csv

26


**Usuários do Windows**: se os comandos acima ou abaixo não funcionarem para você (depende da sua configuração), use este em vez disso:

```python
with open('data/earthquakes.csv', 'r') as file:
    print(len(file.readline().split(',')))
```

Como sabemos que a primeira linha do arquivo contém cabeçalhos e que o arquivo é separado por vírgulas, também podemos contar as colunas usando `head` para obter os cabeçalhos e parseá-los em Python:

In [18]:
headers = !head -n 1 data/earthquakes.csv
len(headers[0].split(','))

26

**Usuários do Windows**: se você precisou usar as alternativas acima, considere experimentar o [Cygwin](https://www.cygwin.com) ou o [Windows Subsystem for Linux (WSL)](https://docs.microsoft.com/en-us/windows/wsl/about).

### Lendo o arquivo
Nosso arquivo é pequeno, tem cabeçalhos na primeira linha e é separado por vírgulas, então não precisamos fornecer nenhum argumento adicional para ler o arquivo com `pd.read_csv()`, mas certifique-se de verificar a [documentação](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) para possíveis argumentos:

In [20]:
df = pd.read_csv('data/earthquakes.csv', sep=',')
df.head(2)

,alert,cdi,code,detail,dmin,felt,gap,ids,mag,magType,...,sources,status,time,title,tsunami,type,types,tz,updated,url
0,NaN,NaN,37389218,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.008693,NaN,85.0,",ci37389218,",1.35,ml,...,",ci,",automatic,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475395144,https://earthquake.usgs.gov/earthquakes/eventp...
1,NaN,NaN,37389202,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.020030,NaN,79.0,",ci37389202,",1.29,ml,...,",ci,",automatic,1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475253925,https://earthquake.usgs.gov/earthquakes/eventp...


Observe que também podemos passar uma URL. Vamos ler o mesmo arquivo do GitHub:

In [21]:
df = pd.read_csv(
    'https://github.com/gstw7/'
    'hands-on-data-analysis-with-pandas-2nd-edition-pt-br'
    '/blob/master/ch_02/data/earthquakes.csv?raw=True'
)
df.head(2)

,alert,cdi,code,detail,dmin,felt,gap,ids,mag,magType,...,sources,status,time,title,tsunami,type,types,tz,updated,url
0,NaN,NaN,37389218,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.008693,NaN,85.0,",ci37389218,",1.35,ml,...,",ci,",automatic,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475395144,https://earthquake.usgs.gov/earthquakes/eventp...
1,NaN,NaN,37389202,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.020030,NaN,79.0,",ci37389202,",1.29,ml,...,",ci,",automatic,1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475253925,https://earthquake.usgs.gov/earthquakes/eventp...


O Pandas geralmente é muito bom em descobrir quais opções usar com base nos dados de entrada, então muitas vezes não precisamos adicionar argumentos à chamada; no entanto, existem muitas opções disponíveis caso precisemos delas, algumas das quais incluem o seguinte:

| Parâmetro | Propósito |
| --- | --- |
| `sep` | Especifica o delimitador |
| `header` | Número da linha onde os nomes das colunas estão localizados; a opção padrão faz com que o `pandas` infera se eles estão presentes |
| `names` | Lista de nomes das colunas a serem usados como cabeçalho |
| `index_col` | Coluna a ser usada como índice |
| `usecols` | Especifica quais colunas ler |
| `dtype` | Especifica os tipos de dados das colunas |
| `converters` | Especifica funções para converter dados em certas colunas |
| `skiprows` | Linhas a serem puladas |
| `nrows` | Número de linhas a serem lidas de cada vez (combine com `skiprows` para ler um arquivo aos poucos) |
| `parse_dates` | Converte automaticamente colunas que contêm datas em objetos datetime |
| `chunksize` | Para ler o arquivo em pedaços |
| `compression` | Para ler arquivos compactados sem extrair antes |
| `encoding` | Especifica a codificação do arquivo |

## Escrevendo um Objeto `DataFrame` em um Arquivo CSV
Observe que o índice do `df` são apenas números de linha, então não queremos mantê-lo. Portanto, passamos `index=False` para o método `to_csv()`:


In [22]:
df.to_csv('output.csv', index=False)

## Escrevendo um Objeto `DataFrame` em um Banco de Dados
Observe o parâmetro `if_exists`. Por padrão, ele dará um erro se você tentar escrever uma tabela que já existe. Aqui, não nos importamos se ela for sobrescrita. Por último, se estivermos interessados em adicionar novas linhas, configuramos para `'append'`.

In [23]:
import sqlite3

with sqlite3.connect('data/quakes.db') as connection:
    pd.read_csv('data/tsunamis.csv').to_sql(
        'tsunamis', connection, index=False, if_exists='replace'
    )

## Criando um Objeto `DataFrame` Consultando um Banco de Dados

Usando um banco de dados SQLite. Caso contrário, você precisa instalar o [SQLAlchemy](https://www.sqlalchemy.org/).

In [24]:
import sqlite3

with sqlite3.connect('data/quakes.db') as connection:
    tsunamis = pd.read_sql('SELECT * FROM tsunamis', connection)

tsunamis.head()

,alert,type,title,place,magType,mag,time
0,None,earthquake,"M 5.0 - 165km NNW of Flying Fish Cove, Christm...","165km NNW of Flying Fish Cove, Christmas Island",mww,5.0,1539459504090
1,green,earthquake,"M 6.7 - 262km NW of Ozernovskiy, Russia","262km NW of Ozernovskiy, Russia",mww,6.7,1539429023560
2,green,earthquake,"M 5.6 - 128km SE of Kimbe, Papua New Guinea","128km SE of Kimbe, Papua New Guinea",mww,5.6,1539312723620
3,green,earthquake,"M 6.5 - 148km S of Severo-Kuril'sk, Russia","148km S of Severo-Kuril'sk, Russia",mww,6.5,1539213362130
4,green,earthquake,"M 6.2 - 94km SW of Kokopo, Papua New Guinea","94km SW of Kokopo, Papua New Guinea",mww,6.2,1539208835130


<hr>
<div>
    <a href="./1-pandas_data_structures.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./3-making_dataframes_from_api_requests.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>